In [5]:
class RequireToString(type):
    def __new__(cls, name, bases, dct):
        if '__str__' not in dct:
            raise TypeError(f"Class '{name}' must define a '__str__' method.")
        return super().__new__(cls, name, bases, dct)

class Person(metaclass=RequireToString):
    def __init__(self, name):
        self.name = name

    def __str__(self):
        return f"Person: {self.name}"

p = Person("Prathiksha")
print(p)



Person: Prathiksha


In [2]:
class AutoInit(type):
    def __new__(cls, name, bases, dct):
        # Check if 'init_fields' exists and is a list
        init_fields = dct.get('init_fields')
        if init_fields is None:
            raise TypeError(f"Class '{name}' must define 'init_fields' as a list.")
        if not isinstance(init_fields, list):
            raise TypeError(f"'init_fields' in class '{name}' must be a list.")

        # Inject __init__ only if it's not already defined
        if '__init__' not in dct:
            def __init__(self, *args, **kwargs):
                # Assign positional arguments
                for field, value in zip(init_fields, args):
                    setattr(self, field, value)
                # Assign keyword arguments if provided
                for field in init_fields[len(args):]:
                    setattr(self, field, kwargs.get(field, None))
            dct['__init__'] = __init__

        return super().__new__(cls, name, bases, dct)

# Example class using AutoInit
class Employee(metaclass=AutoInit):
    init_fields = ['name', 'salary']

    def __str__(self):
        return f"Employee: {self.name}, Salary: {self.salary}"

# Working with positional arguments
e = Employee("Bob", 50000)
print(e)  # Output: Employee: Bob, Salary: 50000

# Working with keyword arguments
e2 = Employee(name="Alice", salary=60000)
print(e2)  # Output: Employee: Alice, Salary: 60000

# Example where an error will be raised
try:
    class InvalidEmployee(metaclass=AutoInit):
        init_fields = "not_a_list"  # This will raise an error
except TypeError as e:
    print(e)



Employee: Bob, Salary: 50000
Employee: Alice, Salary: 60000
'init_fields' in class 'InvalidEmployee' must be a list.


In [11]:
class NegativeNumException(Exception):
    def __init__(self, message="Factorial is not defined for negative numbers."):
        self.message = message
        super().__init__(self.message)


def nfactorial(n):
    if n < 0:
        raise NegativeNumException()
    result = 1
    for i in range(1, n + 1):
        result *= i
    return result


try:
    print(nfactorial(5))    
    print(nfactorial(-3))   
except NegativeNumException as e:
    print(f"Error: {e}")


120
Error: Factorial is not defined for negative numbers.
